In [15]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [16]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=["ARG", 'best_solution_probs', 'classcial', 'run_times'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSolver ', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
# method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["ARG",'best_solution_probs', 'classcial','run_times'], method_order]))

pivot_df

ARG                               \
                           ChocoSolver  QtoSolver QtoSimplifySolver   
pkid variables constraints                                            
0    6         3              0.095949   0.006966          0.011123   
1    15        8              0.294871   0.136345          0.133791   
2    21        12             0.360775   0.198659          0.199832   
3    28        15             0.571255   0.618630          0.653707   
4    12        6              0.215230   0.180411          0.180615   
5    15        9              0.608376   0.387764          0.228525   
6    20        8              1.100469   0.396682          0.389243   
7    24        12             1.530496   1.058158          0.759606   
8    8         6              0.319811   0.043482          0.033675   
9    15        8              0.945742   0.859827          0.860032   
10   18        9              1.391617   1.222445          1.221257   
11   21        10             3.501929   2.924078          2.926642   
12   7         4              0.063660   0.007787          0.009018   
13   10        5              0.422540   0.292581          0.287797   
14   14        6              0.538621   0.376701          0.372690   
15   18        7              0.744384   0.636261          0.637039   
16   9         4              3.936367   0.227036          0.108716   
17   12        5              7.268241   0.644915          0.617482   
18   16        6             22.096429   6.948732          6.721675   
19   20        7             26.214772  23.310687         21.695573   

                                                    best_solution_probs  \
                           QtoSimplifyDiscardSolver         ChocoSolver   
pkid variables constraints                                                
0    6         3                           0.008403           88.339844   
1    15        8                           0.115298           37.721680   
2    21        12                          0.138274           23.228516   
3    28        15                          0.526405           15.750977   
4    12        6                           0.146504           54.845703   
5    15        9                           0.433982           34.536133   
6    20        8                           0.457047           10.239258   
7    24        12                          0.933495            5.780273   
8    8         6                           0.066786           85.882353   
9    15        8                           0.887477           30.565430   
10   18        9                           1.211682           17.415039   
11   21        10                          3.163162            7.282941   
12   7         4                           0.005512           91.709961   
13   10        5                           0.281008           52.784180   
14   14        6                           0.346194           33.178711   
15   18        7                           0.551791           13.045898   
16   9         4                           0.241201           67.864258   
17   12        5                           0.695985           36.614258   
18   16        6                           4.024189           17.494141   
19   20        7                          16.518567           13.665039   

                                                         \
                            QtoSolver QtoSimplifySolver   
pkid variables constraints                                
0    6         3            97.750000         96.520508   
1    15        8            71.078125         71.391602   
2    21        12           53.359375         51.900391   
3    28        15           15.689453         13.142578   
4    12        6            55.908203         55.898438   
5    15        9            48.705078         64.089844   
6    20        8            37.964844         39.125000   
7    24        12           20.062500         32.793945   
8    8         6          

In [17]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [18]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):6.03f}" for x in row]
    print(" & ".join(formatted_row))


 0.096 &  0.007 &  0.008 &  1.029 &  0.693 &  0.407
 0.295 &  0.136 &  0.115 &  5.404 & 23.071 & 12.889
 0.361 &  0.199 &  0.138 &  9.936 & 48.622 & 36.205
 0.571 &  0.619 &  0.526 & 26.186 & 203.721 & 165.529
 0.215 &  0.180 &  0.147 &  3.589 &  5.498 &  1.207
 0.608 &  0.388 &  0.434 &  2.728 &  4.966 &  0.816
 1.100 &  0.397 &  0.457 & 21.431 & 32.510 & 16.321
 1.530 &  1.058 &  0.933 & 14.413 & 29.449 & 11.091
 0.320 &  0.043 &  0.067 &  1.967 &  1.157 &  0.633
 0.946 &  0.860 &  0.887 &  5.403 & 13.117 &  1.801
 1.392 &  1.222 &  1.212 & 13.625 & 53.444 & 46.346
 3.502 &  2.924 &  3.163 & 19.589 & 118.681 & 90.572
 0.064 &  0.008 &  0.006 &  1.379 &  0.914 &  0.462
 0.423 &  0.293 &  0.281 &  2.591 &  4.624 &  1.166
 0.539 &  0.377 &  0.346 &  6.691 & 25.968 & 15.763
 0.744 &  0.636 &  0.552 & 10.564 & 51.179 & 26.028
 3.936 &  0.227 &  0.241 &  3.377 &  7.653 &  3.293
 7.268 &  0.645 &  0.696 & 10.002 & 29.403 & 16.528
22.096 &  6.949 &  4.024 & 37.482 & 176.870 & 105.628
26.215 

In [19]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [20]:
# 提取所需列
best_solution_probs_Qto_discard = pivot_df['best_solution_probs']['QtoSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_Qto_discard = pivot_df['ARG']['QtoSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_Qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_Qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_Qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_Qto_discard.mean()

# 输出结果
print(f"行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): 1.63
行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): 4.12
列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): 1.35
列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): 2.35


In [21]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [22]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [23]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

0.10 & 0.01 & 0.01 & 0.01 & 88.34 & 97.75 & 96.52 & 96.69 & 1.03 & 0.69 & 0.69 & 0.41 & 119 & 137 & 136 & 69.52
0.29 & 0.14 & 0.13 & 0.12 & 37.72 & 71.08 & 71.39 & 67.70 & 5.40 & 23.07 & 22.20 & 12.89 & 121 & 453 & 453 & 463
0.36 & 0.20 & 0.20 & 0.14 & 23.23 & 53.36 & 51.90 & 60.34 & 9.94 & 48.62 & 46.01 & 36.20 & 124 & 453 & 453 & 546
0.57 & 0.62 & 0.65 & 0.53 & 15.75 & 15.69 & 13.14 & 21.55 & 26.19 & 204 & 189 & 166 & 121 & 453 & 453 & 655
0.22 & 0.18 & 0.18 & 0.15 & 54.85 & 55.91 & 55.90 & 64.12 & 3.59 & 5.50 & 5.50 & 1.21 & 118 & 431 & 432 & 150
0.61 & 0.39 & 0.23 & 0.43 & 34.54 & 48.71 & 64.09 & 38.98 & 2.73 & 4.97 & 6.13 & 0.82 & 114 & 449 & 453 & 113
1.10 & 0.40 & 0.39 & 0.46 & 10.24 & 37.96 & 39.12 & 32.19 & 21.43 & 32.51 & 31.05 & 16.32 & 116 & 453 & 453 & 491
1.53 & 1.06 & 0.76 & 0.93 & 5.78 & 20.06 & 32.79 & 24.22 & 14.41 & 29.45 & 40.09 & 11.09 & 118 & 460 & 453 & 346
0.32 & 0.04 & 0.03 & 0.07 & 85.88 & 96.83 & 96.83 & 95.50 & 1.97 & 1.16 & 1.20 & 0.63 & 117 & 142 & 144 & 8

In [24]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [25]:
pivot_df['ARG']

,,,ChocoSolver,QtoSolver,QtoSimplifySolver,QtoSimplifyDiscardSolver
pkid,variables,constraints,,,,
0,6,3,0.095949,0.006966,0.011123,0.008403
1,15,8,0.294871,0.136345,0.133791,0.115298
2,21,12,0.360775,0.198659,0.199832,0.138274
3,28,15,0.571255,0.618630,0.653707,0.526405
4,12,6,0.215230,0.180411,0.180615,0.146504
5,15,9,0.608376,0.387764,0.228525,0.433982
6,20,8,1.100469,0.396682,0.389243,0.457047
7,24,12,1.530496,1.058158,0.759606,0.933495
8,8,6,0.319811,0.043482,0.033675,0.066786


In [26]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df